# Übung zu Kapitel 3.5 - Demonstration von Overfitting und Underfitting
*Eine Übung zum Buch "[Basiswissen KI-Testen - Qualität von und mit KI-basierten Systemen](https://dpunkt.de/produkt/basiswissen-ki-testen/)", ISBN 978-3-86490-947-4*

In dieser Übung wollen wir dir das Prinzip von **Over-** und **Underfitting** anhand einprägsamer und anschaulicher Beispiele demonstrieren. Dafür verwenden wir einen kleinen Datensatz und einfache KI-Modelle. Der verwendete Datensatz enthält lediglich ein Eingabemerkmal und ein Ausgabemerkmal. So können wir den Datensatz in einem zweidimensionalen Plot darstellen und mit bloßem Auge das Muster in den Daten erkennen. Bei den verwendeten KI-Modellen handelt es sich um **Regressionsmodelle**. Grundwissen über [Polynomfunktionen](https://de.wikipedia.org/wiki/Polynom) reicht dabei aus, um das Verhalten der Modelle nachvollziehen zu können. 

*Hinweis:* Auch wenn die Verwendung von Regressionsmodellen banal erscheint und ihnen häufig die Magie der Künstlichen Intelligenz aberkannt wird, sind diese gängige Modelle im Maschinellen Lernen. Wenn wir anhand der folgenden Übung das Prinzip von Over- und Underfitting verstanden haben, können wir dieses Prinzip auf beliebig komplexe Datensätze und ML-Modelle übertragen.

Die Übung ist in drei Schritte eingeteilt:
1. Aufgabe 1: Vorbereitung und Trainingsdatensatz
2. Aufgabe 2a: Regressionsmodelle ausprobieren
3. Aufgabe 2b: Regressionsmodelle für andere Trainingsdaten ausprobieren


## Aufgabe 1: Vorbereitung

**Laden der Funktionsbibliotheken**

In dieser und in den folgenden Übungen verwenden wir *Funktionen aus der freien Software-Bibliothek scikit-learn*, auch abgekürzt sklearn, die zum maschinellen Lernen mit der Programmiersprache Python häufig Verwendung findet. Die in der scikit-learn-Bibliothek bereitgestellten Funktionen sowie Funktionen aus den Bibliotheken *numpy*, *pandas* und *matplotlib* helfen uns in dieser und in den folgenden Übungen bei der Implementierung unserer ML-Modelle und der Visualisierung der Daten.

[<img src="https://numpy.org/doc/stable/_static/numpylogo.svg" alt="Numpy" width="80" height="24">](https://numpy.org/doc/stable/reference/index.html#reference)
&emsp; [<img src="https://pandas.pydata.org/docs/_static/pandas.svg" alt="pandas" width="80" height="24">](https://pandas.pydata.org/docs/reference/index.html)
&emsp; [<img src="https://matplotlib.org/_static/logo_light.svg" alt="Matplotlib" width="100" height="24">](https://matplotlib.org/)
&emsp; [<img src="https://scikit-learn.org/stable/_static/scikit-learn-logo-small.png" alt="Scikit-learn" width="80" height="24">](https://scikit-learn.org/stable/modules/classes.html)


In [ ]:
# Laden aller nötigen Bibliothekn 
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

**Erstellen eines Datensatzes mit einem quadratischen Zusammenhang zwischen Eingaben und Ausgaben**

Als erstes generieren wir einen Datensatz, mit den Eingabewerten *X* und den dazugehörigen Ausgabewerten *y*. Diese sollen einen *quadratischen*
Zusammenhang haben. So wie häufig in der Realität, sind auch unsere Ausgabewerte *y* nicht ganz perfekt, sondern variieren in den Ausgabedaten durch eine zufällige Abweichung. Dies können wir im folgenden Plot sehen.

In [ ]:
# Wir definieren eine quadratische Funktion, die den Zusammenhang zwischen Eingang (X) und Rückgabewert darstellt:
def f(x):
   return (x-5)**2

# Wir generieren unsere Daten als Eingabewerte (X) und Ausgabewerte (y)
X = np.arange(0,10.5,0.5)                  # den Wertebereich für die Eingabewerte (X) festlegen
np.random.seed(42)                         # den Startwert des Zufallsgenerators festlegen
y = f(X) + np.random.normal(0,1.5,len(X))  # Ausgabewerte (y) aus f und addierten Zufallszahlen (Rauschen) erzeugen

# Die folgenden Zeilen erzeugen ein Diagramm mit dem gerade erzeugten Datensatz als Punkte und der
# zugrundeliegenden quadratischen Funktion als Linie, die den idealen Zusammenhang zeigen soll.
plt.plot(X, f(X), color='gray', label="Idealer quadratischer Zusammenhang")
plt.scatter(X,y, label="Datenpunkte")
plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.show()

**Aufteilen der Daten in Trainings- und Testdaten**

Im Folgenden wollen wir Modelle trainieren, die das Muster in den Daten möglichst gut erkennen und möglichst nahe an den idealen quadratischen Zusammenhang herankommen. Bevor wir ein Modell trainieren, teilen wir unseren Datensatz in die Trainingsdaten (*X_train* und *y_train*)  und Testdaten (*X_test* und *y_test*) auf. Dafür verwenden wir die die scikit-learn-Funktion [*train_test_split*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

*Hinweis:* Mehr zum Aufteilen von Trainings- und Testdaten kannst du in der Übung 4.2 erfahren. Hier reicht es, nur die Funktion anzuwenden.

In [ ]:
# Wir teilen Eingabe- (X) und Ausgabedaten (y) in jeweils 70% Trainings- und 30% Testdaten auf:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
plt.scatter(X_train,y_train, label = "Trainingsdaten")
plt.scatter(X_test,y_test, facecolors='none', edgecolors='gray', label="Testdaten")
plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.show()

**Training verschiedener Modelle vorbereiten**

Wir erstellen gleich mehrere Regressions-Modelle mit unterschiedlicher Komplexität, um zu beobachten, wie gut sich die verschiedenen Modelle an das Muster in den Daten anpassen. Dafür implementieren wir die Funktion `Modell_Regression` mit dem Parameter `grad`, der den Grad der Polynomfunktion darstellt. Je größer du den Parameter `grad` wählst, desto 'komplexer' wird das Modell.
Zum Trainieren der Modelle verwenden wir die numpy-Funktion [*polyfit*](https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html), die das am besten zu den Trainingsdaten passende Modell für den angegeben Grad ermittelt. 
Diese Funktion werden wir in den nächsten Aufgaben verwenden.

In [ ]:
# Definition einer Funktion zum Regressionsmodell-Training und grafische Anzeige des trainierten Modells
def Modell_Regression(grad):
    p = np.poly1d(np.polyfit(X_train,y_train,grad))
    plt.scatter(X_train,y_train,label="Trainingsdaten")
    plt.scatter(X_test,y_test, facecolors='none', edgecolors='gray', label="Testdaten")    
    curve_x = np.arange(min(X) -1,max(X)+1,0.01)
    plt.plot(curve_x,p(curve_x),label="Polynomialen Regression vom Grad {}".format(grad))
    plt.xlim((np.min(X)-0.5,np.max([np.max(X), np.max(curve_x)])+0.5))
    plt.ylim((np.min(y)-1.5,np.max(y)+1.5))
    plt.xlabel("X")
    plt.ylabel("y")
    plt.legend()
    plt.plot()
    return np.poly1d(np.polyfit(X_train,y_train,grad))

## Aufgabe 2a: Regressionsmodelle ausprobieren
Probiere nun verschiedene Polynomfunktionen aus, indem du für den Parameter `grad = ...` verschiedene Werte (z.B. 0,1,2,3,4, etc.) setzt, und beobachte dabei, wie gut sich das Modell an die Trainingsdaten anpasst! Wie gut werden die Testdaten von den Modellen abgeschätzt?

In [ ]:
Modell_Regression(grad = ...);

In [ ]:
# Um die Lösung anzuzeigen, bitte diese Zelle zweimal ausführen
%load Lösungen/Lösung01.py

Wenn du ein Gefühl für das Verhalten der Modelle mit unterschiedlichem Grad der Polynomfunktion entwickelt hast, wollen wir gemeinsam das Verhalten der verschiedenen Modelle beleuchten.

**Modell mit Polynomfunktion vom Grad 0**

Als erstes schauen wir ein sehr einfaches Modell mit *grad* = 0 an, also eine Konstante als Ausgabe.

In [ ]:
Modell_Regression(grad = 0);

**Frage:** Wie gut passt sich das Modell mit Grad 0 an die Trainingsdaten und an die Testdaten an? Handelt es sich um *Overfitting* oder um *Underfitting*? Warum passt sich das Modell so schlecht an die Daten an? 

***Lösung:** Klicke auf die folgende Zelle (...) um sie sichtbar zu machen*

Das Modell passt sich sowohl an die Trainings- als auch an die Testdaten nur unzureichend an. Daher handelt es sich um ein **Underfitting**. Grund dafür ist, dass das Modell zu simpel gewählt ist, um das Muster in den Trainingsdaten widerzuspiegeln.

**Modell mit Polynomfunktion vom Grad 1**

In [ ]:
Modell_Regression(grad = 1);

**Frage:** Wie gut passt sich das Modell an die Trainingsdaten und an die Testdaten an? Handelt es sich um *Overfitting* oder um *Underfitting*? Warum passt sich das Modell so schlecht an die Daten an?

***Lösung:** Klicke auf die folgende Zelle (...) um sie sichtbar zu machen*

Während sich das Modell gut an die Trainingsdaten anpasst, spiegelt es den Zusammenhang in den Testdaten gar nicht wider. Daher handelt es sich hier um **Overfitting**. Grund dafür ist, dass der Trainingssatz nicht genügend Datenpunkte und Informationen enthält, um das tatsächliche Muster in den Daten darzustellen.

**Modell mit Polynomfunktion vom Grad 2**

Mit etwas Wissen über den Datensatz kann man vor dem Modelltraining die Annahme treffen, dass wir ein Modell verwenden sollten, dass einen quadratischen Zusammenhang darstellen kann.

In [ ]:
Modell_Regression(grad = 2);

**Frage:** Wie gut passt sich das Modell an die Trainingsdaten und an die Testdaten an? Handelt es sich um *Overfitting* oder um *Underfitting*? Warum passt sich das Modell so schlecht an die Daten an?

***Lösung:** Klicke auf die folgende Zelle (...) um sie sichtbar zu machen*

Obwohl wir ein Modell gewählt haben, das den Zusammenhang in den Daten (grob) widerspiegelt, hat sich das Modell immer noch besser an die Trainingsdaten als an die Testdaten angepasst. Daher sprechen wir auch hier von einem **Overfitting**. Grund für das Overfitting ist, dass der Trainingsdatensatz nicht genügend Daten enthält. Der Datensatz ist so aufgeteilt, dass im Trainingsdatensatz zu wenig der entscheidenden Informationen über den Zusammenhang zwischen Eingabedaten X und Ausgabedaten y enthalten sind. Der rechte Arm der Parabel ist nicht ausreichend in den Trainingsdaten repräsentiert.

## Aufgabe 2b: Regressionsmodelle mit anderen Trainingsdaten ausprobieren
Für ein erneutes Training teilen wir die Daten *zufällig* in die Trainings- und Testdaten auf. Setze dafür in der folgenden Funktion [*train_test_split*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) den Parameter *shuffle* (auf Deutsch *mischen*) auf *True*.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=..., random_state=42)

In [ ]:
# Um die Lösung anzuzeigen, bitte diese Zelle zweimal ausführen
%load Lösungen/Lösung02.py

In [ ]:
plt.scatter(X_train,y_train, label = "Trainingsdaten")
plt.scatter(X_test,y_test, facecolors='none', edgecolors='gray', label="Testdaten")
plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.show()

Wir sehen, dass die Daten nun eher zufällig in Trainings- und Testdaten aufgeteilt sind. Nun enthalten die Trainingsdaten auch Informationen über den rechten Arm der Parabel.

**Erneutes Training verschiedener Modelle**

Mit der neuen Aufteilung der Daten in Trainings- und Testdaten wollen wir erneut beobachten, wie gut die Modelle das Muster in den Daten erkennen.

Probiere verschiedene Polynomfunktionen aus, indem du für den Parameter `grad = ...` verschiedene Werte (z.B. 0, 1, 2, 3, 4, etc.) einsetzt, und beobachte dabei, wie gut sich das Modell an die Trainingsdaten anpasst! Wie gut werden die Testdaten von den Modellen abgeschätzt?

In [ ]:
Modell_Regression(grad = ...);

In [ ]:
# Um die Lösung anzuzeigen, bitte diese Zelle zweimal ausführen
%load Lösungen/Lösung03.py

**Modell mit Polynomfunktion von Grad 1**

Nun wollen wir gemeinsam das Verhalten der verschiedenen Modelle bei der zufälligen Aufteilung in Test- und Trainingsdaten beleuchten. Als erstes schauen wir uns wieder ein sehr einfaches Modell an mit *grad* = 1.

In [ ]:
Modell_Regression(grad = 1);

**Frage:** Wie gut passt sich das Modell an die Trainingsdaten und an die Testdaten an? Handelt es sich um Overfitting oder um Underfitting? Warum passt sich das Modell so schlecht an die Daten an?

***Lösung:** Klicke auf die folgende Zelle (...) um sie sichtbar zu machen*

Das Modell passt sich sowohl an die Trainings- als auch an die Testdaten nur unzureichend an. Daher handelt es sich um ein **Underfitting**. Grund dafür ist, dass das Modell zu simpel gewählt ist, um das Muster in den Trainingsdaten widerzuspiegeln. Dieses Verhalten ist unabhängig von der Verteilung der Daten in Test- und Trainingsdaten.

**Modell mit Polynomfunktion vom Grad 2**

In [ ]:
Modell_Regression(grad = 2);

**Frage:** Wie gut passt sich das Modell an die Trainingsdaten und an die Testdaten an?

***Lösung:** Klicke auf die folgende Zelle (...) um sie sichtbar zu machen*

Das Modell stellt das Muster der Daten gut dar, sowohl für die Trainingsdaten als auch für die Testdaten. Die Trainingsdaten spiegeln den tatsächlichen Zusammenhang der Eingabedaten X und der Ausgabe y sehr gut wieder und das Modell ist passend zu dem Muster in den Daten gewählt.

**Modell mit Polynomfunktion vom Grad 7**

In [ ]:
Modell_Regression(grad = 7);

**Frage:** Wie gut passt sich das Modell an die Trainingsdaten und an die Testdaten an?

***Lösung:** Klicke auf die folgende Zelle (...) um sie sichtbar zu machen*

Während sich das Modell sehr gut an Trainingsdaten anpasst, spiegelt das Modell den tatsächlichen Zusammenhang in den Daten nicht wider. Die Testdaten liegen zum Teil weit entfernt von dem Modell. Daher handelt es sich hier um **Overfitting**. Grund dafür ist, dass das Modell zu komplex gewählt wurde. Das Modell hat so das Rauschen in den Trainingsdaten mitgelernt, aber nicht das zugrundeliegende Muster in den Daten erkannt.